In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('../../Data/CSV-files/dataset-of-00sv2.csv')
#df = pd.read_csv('../../Data/CSV-files/dataset-of-90sv2.csv')
#df = pd.read_csv('../../Data/CSV-files/dataset-of-60sv2.csv')

In [ ]:
data = df.sample(frac=1)
data.drop(["track","artist","uri"],axis=1,inplace=True)

In [ ]:
#Alle rader og bare den siste kollonnen
target = data.iloc[:,[15]]
#Henter ut alle rader, og alle kollonner fra kol: 0 til og med nest siste 
#columns_dont_want = ["target"]
#select = [x for x in data.columns if x not in columns_dont_want]
#unscaled_inputs = data.iloc[:,select]

unscaled_inputs = data.loc[:, data.columns != "target"]

In [ ]:
unscaled_inputs.keys()

In [ ]:
#Samler verdiene rundt 0, 
scaled_inputs = preprocessing.scale(unscaled_inputs)

In [ ]:
samples_count = scaled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

In [ ]:
train_inputs = scaled_inputs[:train_samples_count]
train_targets = target[:train_samples_count]

validation_inputs = scaled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = target[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = scaled_inputs[train_samples_count+validation_samples_count:]
test_targets = target[train_samples_count+validation_samples_count:]

In [ ]:
print('train_inputs',train_inputs.shape)
print('train_targets',train_targets.shape)
print('validation_inputs',validation_inputs.shape )
print('validation_targets',validation_targets.shape)
print('test_inputs', test_inputs.shape)
print('test_targets',test_targets.shape)
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

In [ ]:
#Hvorfor lagre det som fil og hente ut verdier slik det gjøre i denne og neste blokk?
np.savez('Spotify_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Spotify_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Spotify_data_test', inputs=test_inputs, targets=test_targets)

In [ ]:
npz = np.load('Spotify_data_train.npz')
#Henter ut fra filer med navnet vi lagret det med over
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Spotify_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Spotify_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [ ]:
# Set the input and output sizes
input_size = 17 # count of features
output_size = 2 # count of targets
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50 # counts of neurons

# define how the model will look like
#Stacker ulike lak med hidden layer "oppå" hverandre i modellen 
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 3nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [ ]:
#SJEKK HVA DE ULIKE HER GJØR, PRØV FORSKJELLIGE 
#Optimizer er gradient descent?

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
#custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# That's where we train the model we have built.
# set the batch size
batch_size = 300
# set a maximum number of training epochs
max_epochs = 10

# fit the model
# note that this time the train, validation and test data are not iterable
history = model.fit(  train_inputs, # train inputs
                      train_targets, # train targets
                      batch_size=batch_size, # batch size
                      epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
                      # callbacks are functions called by a task when a task is completed
                      # task here is to check if val_loss is increasing
                      #callbacks=[early_stopping], # early stopping
                      validation_data=(validation_inputs, validation_targets), # validation data
                      verbose = 2 # making sure we get enough information about the training process
          )  

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

accuracy = history.history['accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, validation_loss, 'b-')
plt.plot(epoch_count,accuracy)
plt.legend(['Training Loss', 'Validation Loss','Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_inputs)

In [ ]:
print(predictions[0])
print(test_targets[0])

In [ ]:
def plot_value_array(i, predictions_array, true_label):
    true_label = test_targets[i]
    plt.grid(False)

    thisplot = plt.bar(range(2), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)
    
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'
    
    plt.ylabel("{:2.0f}%".format(100*np.max(predictions_array),color=color))

    
    thisplot[predicted_label].set_color('red')
    thisplot[true_label[0]].set_color('blue')


In [ ]:
i = 105
plt.figure(figsize=(6,3))
plot_value_array(i, predictions[i],  test_targets)
plt.show()

In [ ]:
num_rows = 10
num_cols = 10
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_targets)
plt.tight_layout()
plt.show()

In [ ]:
wrong_guess = []
for i in range(100):
    index = np.argmax(predictions[i])
    if(index!=test_targets[i]):
        wrong_guess.append(test_inputs[i])
        

In [ ]:
print(np.argmax(predictions[2]))
print(predictions[2])

In [ ]:
#Burde egentlig ha med alle feltene her(mangle 'target','weeks'?)
df = pd.DataFrame(wrong_guess)
df = df.rename(columns={0:'danceability',1:'energy',2:'key',3:'loudness',4:'mode',5:'speechiness',
       6:'acousticness',7:'instrumentalness',8:'liveness',9:'valence',10:'tempo',
       11:'duration_ms',12:'time_signature',13:'chorus_hit',14:'sections',
       15:'artist_popularity',16:'artist_followers'})


In [ ]:
sns.heatmap(df.corr())

In [ ]:
#Lav acousticness, speechness, instrumentalness, liveness, få followers
for i in df.columns:
    plt.hist(df[i])
    plt.title(i)
    plt.show()

In [ ]:
for i in df.columns:
    for j in df.columns:
        sns.scatterplot(x=i ,y=j, data =df)
        plt.show()

In [ ]:
a = [[1,2],[3,5,6,7,8]]
print(np.argmax(a))